In [8]:
!pip install Levenshtein

In [51]:
# prompt: create a get_movie_details function which fetches movie details from omdb

import requests

def get_movie_details(movie_name):
  """
  This function fetches movie details from omdb based on the provided movie name.

  Args:
    movie_name: The name of the movie to fetch details for.

  Returns:
    A dictionary containing the movie details, or None if the request fails.
  """

  try:
    response = requests.get(f"http://www.omdbapi.com/?t={movie_name}&apikey=thewdb")
    if response.status_code == 200:
      return response.json()
    else:
      print(f"Error fetching movie details: {response.status_code}")
      return None
  except Exception as e:
    print(f"Error fetching movie details: {e}")
    return None



In [52]:
import warnings
import requests
warnings.filterwarnings('ignore')
response = requests.get("http://www.omdbapi.com/?s=movie&apikey=d9f6512")

data = response.json()

movie_names = [movie["Title"] for movie in data["Search"]]

import pandas as pd

df = pd.DataFrame(columns=["Title", "Year", "Director", "Cast", "Plot", "Maturity Rating", "Language", "IMDB Rating", "Rotten Tomatoes","Review","Sentiment"])

for movie_name in movie_names:
    movie_details = get_movie_details(movie_name)
    if movie_details:
        df = df.append({
            "Title": movie_details["Title"],
            "Year": movie_details["Year"],
            "Director": movie_details["Director"],
            "Genre": movie_details['Genre'],
            "Cast": movie_details["Actors"],
            "Plot": movie_details["Plot"],
            "Maturity Rating": movie_details["Rated"],
            "Language": movie_details["Language"],
            "IMDB Rating": movie_details["imdbRating"],
            "Rotten Tomatoes": movie_details["Ratings"][1]["Value"],
            "Review": [''],
            "Sentiment": 0
        }, ignore_index=True)






In [53]:
# prompt: write the code such that when we add review the tuple gets updated and the list gets appended instead of making a new row
import Levenshtein
import requests

def get_movies_from_tmdb(query):
  url = "https://api.themoviedb.org/3/search/movie"
  params = {"api_key": "84f349e500e907768a70f6b21a70766f", "query": query}
  response = requests.get(url, params=params)
  data = response.json()
  movies = [movie["title"] for movie in data["results"]]
  return movies

def autocomplete_movie_names(query):
  movies = get_movies_from_tmdb(query)
  distances = [Levenshtein.distance(query, movie) for movie in movies]
  closest_movies = [movies[i] for i in sorted(range(len(movies)), key=lambda i: distances[i])]
  return closest_movies[:5]

def main():
  query = input("Enter a movie name: ")
  autocomplete_movies = autocomplete_movie_names(query)
  return autocomplete_movies[0]




In [27]:
if __name__ == "__main__":
  movie_name=main()

  sentiment = int(input("Enter sentiment (0/1): "))
  review = input("Enter your review: ")

  movie_index = df[df["Title"] == movie_name].index[0]

  df.loc[movie_index, "Review"] = df.loc[movie_index, "Review"] + [review]
  df.loc[movie_index, "Sentiment"] = sentiment

  print("Movie details updated successfully.")


Enter a movie name: simpsons
Enter sentiment (0/1): 1
Enter your review: abc
Movie details updated successfully.


In [28]:
df

,Title,Year,Director,Cast,Plot,Maturity Rating,Language,IMDB Rating,Rotten Tomatoes,Review,Sentiment,Genre
0,The Lego Movie,2014,"Phil Lord, Christopher Miller","Chris Pratt, Will Ferrell, Elizabeth Banks","An ordinary LEGO construction worker, thought ...",PG,"English, Turkish",7.7,96%,"[, abc]",1,"Animation, Action, Adventure"
1,The Simpsons Movie,2007,David Silverman,"Dan Castellaneta, Julie Kavner, Nancy Cartwright","After Homer pollutes the town's water supply, ...",PG-13,English,7.3,87%,"[, abc]",1,"Animation, Adventure, Comedy"
2,El Camino: A Breaking Bad Movie,2019,Vince Gilligan,"Aaron Paul, Jonathan Banks, Matt Jones","Fugitive Jesse Pinkman runs from his captors, ...",TV-MA,"English, Spanish",7.3,92%,[],0,"Action, Crime, Drama"
3,Scary Movie,2000,Keenen Ivory Wayans,"Anna Faris, Jon Abrahams, Marlon Wayans",A year after disposing of the body of a man th...,R,English,6.3,51%,[],0,Comedy
4,The Super Mario Bros. Movie,2023,"Aaron Horvath, Michael Jelenic, Pierre Leduc","Chris Pratt, Anya Taylor-Joy, Charlie Day",A plumber named Mario travels through an under...,PG,English,7.0,59%,[],0,"Animation, Adventure, Comedy"
5,Bee Movie,2007,"Simon J. Smith, Steve Hickner","Jerry Seinfeld, Renée Zellweger, Matthew Brode...","Barry B. Benson, a bee just graduated from col...",PG,English,6.1,50%,[],0,"Animation, Adventure, Comedy"
6,Scary Movie 2,2001,Keenen Ivory Wayans,"Anna Faris, Marlon Wayans, Antony Acker",Four teens are tricked by Professor Oldman (Ti...,R,English,5.3,14%,[],0,"Comedy, Horror"
7,The Lego Batman Movie,2017,Chris McKay,"Will Arnett, Michael Cera, Rosario Dawson",A cooler-than-ever Bruce Wayne must deal with ...,PG,English,7.3,90%,[],0,"Animation, Action, Adventure"
8,Scary Movie 3,2003,David Zucker,"Anna Faris, Charlie Sheen, Regina Hall",Cindy must investigate mysterious crop circles...,PG-13,English,5.5,35%,[],0,Comedy
9,Scary Movie 4,2006,David Zucker,"Anna Faris, Regina Hall, Craig Bierko",Cindy finds out the house she lives in is haun...,PG-13,"English, Japanese",5.1,34%,[],0,Comedy


In [67]:
# prompt: create a function to filter movies based on genres and based on elements stored in the review list

import pandas as pd

def filter_movies_by_review_element(element):
  """
  Filters the movie DataFrame based on the provided review element.

  Args:
    element: The review element to filter by.

  Returns:
    A new DataFrame containing only movies with the specified review element.
  """
  movies_with_element = []
  for i, row in df.iterrows():
    for review in row["Review"]:
      if element in review:
        movies_with_element.append(row)
        break
  return pd.DataFrame(movies_with_element)


In [77]:
review_filter=input("Enter a review a word to filter:")
filter_movies_by_review_element(review_filter)

Enter a review a word to filterabc


""


In [75]:
# prompt: create a function to filter movies based on genre after splitting genre into singular strings without creating new dataframe

def filter_movies_by_genre(element):
  filtered_movies = []
  for i, row in df.iterrows():
    for genre in row["Genre"].split(','):
      if element in genre:
        filtered_movies.append(row)
        break
  return pd.DataFrame(filtered_movies)


In [78]:
genre_filter=input("Enter a genre to filter:")
filter_movies_by_genre(genre_filter)

Enter a genre to filterComedy


,Title,Year,Director,Cast,Plot,Maturity Rating,Language,IMDB Rating,Rotten Tomatoes,Review,Sentiment,Genre
1,The Simpsons Movie,2007,David Silverman,"Dan Castellaneta, Julie Kavner, Nancy Cartwright","After Homer pollutes the town's water supply, ...",PG-13,English,7.3,87%,[],0,"Animation, Adventure, Comedy"
3,Scary Movie,2000,Keenen Ivory Wayans,"Anna Faris, Jon Abrahams, Marlon Wayans",A year after disposing of the body of a man th...,R,English,6.3,51%,[],0,Comedy
4,The Super Mario Bros. Movie,2023,"Aaron Horvath, Michael Jelenic, Pierre Leduc","Chris Pratt, Anya Taylor-Joy, Charlie Day",A plumber named Mario travels through an under...,PG,English,7.0,59%,[],0,"Animation, Adventure, Comedy"
5,Bee Movie,2007,"Simon J. Smith, Steve Hickner","Jerry Seinfeld, Renée Zellweger, Matthew Brode...","Barry B. Benson, a bee just graduated from col...",PG,English,6.1,50%,[],0,"Animation, Adventure, Comedy"
6,Scary Movie 2,2001,Keenen Ivory Wayans,"Anna Faris, Marlon Wayans, Antony Acker",Four teens are tricked by Professor Oldman (Ti...,R,English,5.3,14%,[],0,"Comedy, Horror"
8,Scary Movie 3,2003,David Zucker,"Anna Faris, Charlie Sheen, Regina Hall",Cindy must investigate mysterious crop circles...,PG-13,English,5.5,35%,[],0,Comedy
9,Scary Movie 4,2006,David Zucker,"Anna Faris, Regina Hall, Craig Bierko",Cindy finds out the house she lives in is haun...,PG-13,"English, Japanese",5.1,34%,[],0,Comedy
